In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 로드
print("데이터 로드 중...")
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

print(f"학습 데이터 크기: {train_df.shape}")
print(f"테스트 데이터 크기: {test_df.shape}")
print(f"제출 샘플 크기: {sample_submission.shape}")

# 기본적인 데이터 정보 확인
print("\n학습 데이터 정보:")
print(train_df.info())

print("\n결측치 확인:")
print(train_df.isnull().sum())

# 타겟 변수 분포 확인
print("\n타겟 변수(Cancer) 분포:")
print(train_df['Cancer'].value_counts())
print(f"암 양성 비율: {train_df['Cancer'].mean() * 100:.2f}%")

# 1. EDA & 전처리
print("\n데이터 전처리 진행 중...")

# ID 열 drop
train_df_processed = train_df.drop('ID', axis=1)
test_df_processed = test_df.drop('ID', axis=1)

# 범주형(문자열) 변수와 수치형 변수 분리
categorical_cols = train_df_processed.select_dtypes(include=['object']).columns
numerical_cols = train_df_processed.select_dtypes(exclude=['object']).columns.drop('Cancer')

print(f"\n범주형 변수: {list(categorical_cols)}")
print(f"수치형 변수: {list(numerical_cols)}")

# 2. 개선된 EDA 및 특성 엔지니어링
print("\n개선된 특성 엔지니어링 진행 중...")

# # 2.1 Age 관련 특성 개선
# # 이미지 1과 인사이트를 기반으로 나이 구간화 개선
# # 나이와 암 관계를 더 세밀하게 고려 (개선 #1)
# train_df_processed['Age_Bin'] = pd.cut(
#     train_df_processed['Age'], 
#     bins=[0, 25, 35, 50, 65, 100], 
#     labels=[0, 1, 2, 3, 4]
# ).astype(int)

# test_df_processed['Age_Bin'] = pd.cut(
#     test_df_processed['Age'], 
#     bins=[0, 25, 35, 50, 65, 100], 
#     labels=[0, 1, 2, 3, 4]
# ).fillna(2).astype(int)  # 범위 밖의 값은 중간 연령대로 설정

# # 2.2 의학적 검사 결과의 상호작용 및 비율 추가 (개선 #2)
# # TSH, T3, T4 사이의 관계는 갑상선 기능에 중요한 정보를 제공
# train_df_processed['TSH_T4_Ratio'] = train_df_processed['TSH_Result'] / train_df_processed['T4_Result']
# test_df_processed['TSH_T4_Ratio'] = test_df_processed['TSH_Result'] / test_df_processed['T4_Result']

# train_df_processed['T3_T4_Ratio'] = train_df_processed['T3_Result'] / train_df_processed['T4_Result']
# test_df_processed['T3_T4_Ratio'] = test_df_processed['T3_Result'] / test_df_processed['T4_Result']

# # 갑상선 기능을 나타내는 합성 점수 (개선 #3)
# train_df_processed['Thyroid_Function_Score'] = (
#     (train_df_processed['TSH_Result'] - train_df_processed['TSH_Result'].mean()) / train_df_processed['TSH_Result'].std() +
#     (train_df_processed['T4_Result'] - train_df_processed['T4_Result'].mean()) / train_df_processed['T4_Result'].std() +
#     (train_df_processed['T3_Result'] - train_df_processed['T3_Result'].mean()) / train_df_processed['T3_Result'].std()
# )

# test_df_processed['Thyroid_Function_Score'] = (
#     (test_df_processed['TSH_Result'] - train_df_processed['TSH_Result'].mean()) / train_df_processed['TSH_Result'].std() +
#     (test_df_processed['T4_Result'] - train_df_processed['T4_Result'].mean()) / train_df_processed['T4_Result'].std() +
#     (test_df_processed['T3_Result'] - train_df_processed['T3_Result'].mean()) / train_df_processed['T3_Result'].std()
# )

# # 2.3 결절 크기 비선형 변환 (개선 #4)
# # 이미지 6에서 Nodule_Size가 양쪽 클래스 간에 차이가 적어 보임 - 비선형 변환 도입
# train_df_processed['Nodule_Size_Log'] = np.log1p(train_df_processed['Nodule_Size'])
# test_df_processed['Nodule_Size_Log'] = np.log1p(test_df_processed['Nodule_Size'])

# train_df_processed['Nodule_Size_Squared'] = train_df_processed['Nodule_Size'] ** 2
# test_df_processed['Nodule_Size_Squared'] = test_df_processed['Nodule_Size'] ** 2

# # 2.4 범주형 변수 상호작용 (개선 #5)
# # 이미지 5에서 중요한 특성으로 나타난 Race_ASN 및 Family_Background_Positive 활용
# # 원-핫 인코딩 전에 상호작용 특성 생성
# train_df_processed['Race_Family_Risk'] = ((train_df_processed['Race'] == 'ASN') & 
#                                          (train_df_processed['Family_Background'] == 'Positive')).astype(int)
# test_df_processed['Race_Family_Risk'] = ((test_df_processed['Race'] == 'ASN') & 
#                                         (test_df_processed['Family_Background'] == 'Positive')).astype(int)

# # 2.5 방사선 노출과 연령 상호작용 (개선 #6)
# train_df_processed['Radiation_Age_Risk'] = ((train_df_processed['Radiation_History'] == 'Exposed') & 
#                                            (train_df_processed['Age'] < 40)).astype(int)
# test_df_processed['Radiation_Age_Risk'] = ((test_df_processed['Radiation_History'] == 'Exposed') & 
#                                           (test_df_processed['Age'] < 40)).astype(int)

# # 2.6 건강 위험 요소 결합 (개선 #7)
# train_df_processed['Health_Risk_Score'] = (
#     (train_df_processed['Smoke'] == 'Smoker').astype(int) +
#     (train_df_processed['Weight_Risk'] == 'Obese').astype(int) +
#     (train_df_processed['Diabetes'] == 'Yes').astype(int)
# )
# test_df_processed['Health_Risk_Score'] = (
#     (test_df_processed['Smoke'] == 'Smoker').astype(int) +
#     (test_df_processed['Weight_Risk'] == 'Obese').astype(int) +
#     (test_df_processed['Diabetes'] == 'Yes').astype(int)
# )

# 2.7 아이오딘 결핍과 국가 상호작용 (개선 #8)
# 특정 국가와 아이오딘 결핍 간의 관계가 중요한 특성으로 보임
train_df_processed['Iodine_Country_Risk'] = ((train_df_processed['Iodine_Deficiency'] == 'Deficient') & 
                                            (train_df_processed['Country'] == 'IND')).astype(int)
test_df_processed['Iodine_Country_Risk'] = ((test_df_processed['Iodine_Deficiency'] == 'Deficient') & 
                                           (test_df_processed['Country'] == 'IND')).astype(int)

# 2.8 수치형 변수의 이상치 클리핑 (개선 #9)
# 상자 그림 기반으로 이상치 존재 확인 후 처리
for col in numerical_cols:
    q1 = train_df_processed[col].quantile(0.01)
    q3 = train_df_processed[col].quantile(0.99)
    train_df_processed[col] = train_df_processed[col].clip(q1, q3)
    test_df_processed[col] = test_df_processed[col].clip(q1, q3)

# 범주형 변수를 One-Hot Encoding으로 변환
train_df_encoded = pd.get_dummies(train_df_processed, columns=categorical_cols, drop_first=True)
test_df_encoded = pd.get_dummies(test_df_processed, columns=categorical_cols, drop_first=True)

# One-Hot Encoding 후 열 일치시키기
missing_cols = set(train_df_encoded.columns) - set(test_df_encoded.columns)
for col in missing_cols:
    if col != 'Cancer':  # Cancer 열은 test에 없으므로 제외
        test_df_encoded[col] = 0

# 학습 데이터에 없는 테스트 데이터의 열 제거
extra_cols = set(test_df_encoded.columns) - set(train_df_encoded.columns)
for col in extra_cols:
    test_df_encoded = test_df_encoded.drop(col, axis=1)
        
# 열 순서 일치시키기 (Cancer 열 제외)
test_df_encoded = test_df_encoded[train_df_encoded.columns.drop('Cancer')]

# 새로운 수치형 열 목록 업데이트
numerical_cols = [col for col in train_df_encoded.select_dtypes(include=['float64', 'int64']).columns 
                if col != 'Cancer']

# 학습 데이터와 검증 데이터 분리 (20% 검증 데이터)
X = train_df_encoded.drop('Cancer', axis=1)
y = train_df_encoded['Cancer']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 수치형 변수 정규화
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
test_df_encoded[numerical_cols] = scaler.transform(test_df_encoded[numerical_cols])

# 3. 모델 구축
print("\n모델 구축 중...")

# SMOTE를 사용하여 클래스 불균형 처리
print("SMOTE 적용하여 클래스 불균형 해소...")
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print(f"SMOTE 적용 전 학습 데이터 크기: {X_train.shape}, 양성 비율: {y_train.mean() * 100:.2f}%")
print(f"SMOTE 적용 후 학습 데이터 크기: {X_train_smote.shape}, 양성 비율: {y_train_smote.mean() * 100:.2f}%")

# XGBoost 모델 학습
print("\nXGBoost 모델 학습 중...")
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    scale_pos_weight=1
)

xgb_model.fit(X_train_smote, y_train_smote)

# 검증 데이터로 성능 평가
y_val_pred = xgb_model.predict(X_val)
val_f1 = f1_score(y_val, y_val_pred)

print("\n검증 데이터에 대한 성능 평가:")
print(classification_report(y_val, y_val_pred))
print(f"Validation F1-Score: {val_f1:.4f}")

# 특성 중요도 시각화
plt.figure(figsize=(12, 8))
xgb.plot_importance(xgb_model, max_num_features=20)
plt.title('XGBoost Feature Importance (Top 20)')
plt.tight_layout()
plt.savefig('improved_feature_importance.png')
plt.close()

# 테스트 데이터에 예측
print("\n테스트 데이터에 예측 중...")
test_predictions = xgb_model.predict(test_df_encoded)

# sample_submission 파일에 예측 결과 저장
sample_submission['Cancer'] = test_predictions.astype(int)
sample_submission.to_csv('improved_submission.csv', index=False)

print("\n예측 완료! 'improved_submission.csv' 파일에 결과가 저장되었습니다.")

# 예측 결과 확인
print("예측 결과 분포:")
print(sample_submission['Cancer'].value_counts())
print(f"암 양성 예측 비율: {sample_submission['Cancer'].mean() * 100:.2f}%")

데이터 로드 중...
학습 데이터 크기: (87159, 16)
테스트 데이터 크기: (46204, 15)
제출 샘플 크기: (46204, 2)

학습 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87159 entries, 0 to 87158
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 87159 non-null  object 
 1   Age                87159 non-null  int64  
 2   Gender             87159 non-null  object 
 3   Country            87159 non-null  object 
 4   Race               87159 non-null  object 
 5   Family_Background  87159 non-null  object 
 6   Radiation_History  87159 non-null  object 
 7   Iodine_Deficiency  87159 non-null  object 
 8   Smoke              87159 non-null  object 
 9   Weight_Risk        87159 non-null  object 
 10  Diabetes           87159 non-null  object 
 11  Nodule_Size        87159 non-null  float64
 12  TSH_Result         87159 non-null  float64
 13  T4_Result          87159 non-null  float64
 14  T3_Result          87159 n

<Figure size 1200x800 with 0 Axes>